In [1]:
%load_ext autoreload
%autoreload 2

# MATH data generation pipeline


## Imports and Setup


In [2]:
from utils.template_utils import *
from utils.file_utils import *
from utils.generation_utils import *
from utils.sample_utils import *

from openai import OpenAI
import numpy as np
from pathlib import Path

np.random.seed(42)

In [3]:
train_dir = Path("../data/train")
train_sample_dir = Path("../data/train_sample")
train_prepared_data_dir = Path("../data/train_prepared")
train_output_dir = Path("../data/train_generated")

train_topic_dirs = list(train_dir.glob("*"))
train_topics = list(map(lambda x: x.name, train_topic_dirs))

In [4]:
client = OpenAI()

## Sample Data


In [5]:
sample_size = 75

for topic_dir in train_topic_dirs:
    sample_dir = train_sample_dir / topic_dir.name
    sample_files(topic_dir, sample_dir, sample_size=sample_size)

## Prepare Data


In [40]:
for topic in train_topics:
    input_path = train_sample_dir / topic
    output_path = train_prepared_data_dir / f"{topic}.jsonl"

    raw2prepared(input_path, output_path)

## Generate Responses


In [44]:
for topic in train_topics:
    topic_file = topic + ".jsonl"

    input_path = train_prepared_data_dir / topic_file
    output_path = train_output_dir / topic_file
    if output_path.exists():
       output_path.unlink()
       
    input_path_str = "/".join(input_path.parts)
    output_path_str = "/".join(output_path.parts)

    !python parallel_processing.py \
        --requests_filepath {input_path_str} \
        --save_filepath {output_path_str} \
        --request_url https://api.openai.com/v1/chat/completions \
        --max_requests_per_minute 400 \
        --max_tokens_per_minute 50000 \
        --token_encoding_name cl100k_base \
        --max_attempts 5 \
        --logging_level 20

INFO:root:Starting request #0
INFO:root:Starting request #1
INFO:root:Starting request #2
INFO:root:Starting request #3
INFO:root:Starting request #4
INFO:root:Starting request #5
INFO:root:Starting request #6
INFO:root:Starting request #7
INFO:root:Starting request #8
INFO:root:Starting request #9
INFO:root:Starting request #10
INFO:root:Starting request #11
INFO:root:Starting request #12
INFO:root:Starting request #13
INFO:root:Starting request #14
INFO:root:Starting request #15
INFO:root:Starting request #16
INFO:root:Starting request #17
INFO:root:Starting request #18
INFO:root:Starting request #19
INFO:root:Starting request #20
INFO:root:Starting request #21
INFO:root:Starting request #22
INFO:root:Starting request #23
INFO:root:Starting request #24
INFO:root:Starting request #25
INFO:root:Starting request #26
INFO:root:Starting request #27
INFO:root:Starting request #28
INFO:root:Starting request #29
INFO:root:Starting request #30
INFO:root:Starting request #31
INFO:root:Starting

In [47]:
for topic in train_topics:
    topic_file = topic + ".jsonl"
    output_path = train_output_dir / topic_file
    print(f"{topic} errors: {count_errors(output_path)}")

counting_and_probability errors: 0
intermediate_algebra errors: 0
number_theory errors: 0
precalculus errors: 0
prealgebra errors: 0
geometry errors: 0
algebra errors: 0


## Evaluate Responses


In [6]:
for topic in train_topics:
    topic_file = topic + ".jsonl"
    evaluate(train_sample_dir / topic, train_output_dir / topic_file)

In [8]:
for topic in train_topics:
    topic_file = topic + ".jsonl"
    metrics = get_metrics(train_output_dir / topic_file)
    print(f"{topic}: {metrics}")

counting_and_probability: {'correct': 22, 'incorrect': 53, 'accuracy': 0.29333333333333333}
intermediate_algebra: {'correct': 17, 'incorrect': 58, 'accuracy': 0.22666666666666666}
number_theory: {'correct': 26, 'incorrect': 49, 'accuracy': 0.3466666666666667}
precalculus: {'correct': 20, 'incorrect': 55, 'accuracy': 0.26666666666666666}
prealgebra: {'correct': 50, 'incorrect': 25, 'accuracy': 0.6666666666666666}
geometry: {'correct': 21, 'incorrect': 54, 'accuracy': 0.28}
algebra: {'correct': 42, 'incorrect': 33, 'accuracy': 0.56}
